In [1]:
import pandas as pd
import sys,json,pickle,re
from utils import *
from config import *

Original file downloaded from Github (https://github.com/t-davidson/hate-speech-and-offensive-language)

In [2]:
train_file = "data/labeled_data.csv"

Load Train data in dataframe

In [3]:
train_df = pd.read_csv(train_file)
print(train_df.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


Find Unique labels and create label map

In [4]:
# 0 - hate speech 1 - offensive language 2 - neither as found in documentation
labels = list(train_df["class"].unique())
print(labels)
label2ind = {"hate_speech":0,"offensive_language":1,"neither":2}
ind2label = {i:l for l,i in label2ind.items()}
with open("data/label2ind.json","w") as f:
    json.dump(label2ind,f,indent=1)
with open("data/ind2label.json","w") as f:
    json.dump(ind2label,f,indent=1)

[2, 1, 0]


Create a new dataframe with only text and label columnn, also dump in CSV

In [5]:
new_train_df = train_df[["tweet","class"]]
new_train_df["class"] = new_train_df["class"].map(lambda x:ind2label[x])
new_train_df["tweet"] = new_train_df["tweet"].map(lambda x:' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|RT"," ",x).split()))
print(new_train_df.head())
train_file_path = "data/hate_speech_train.csv"
new_train_df.to_csv(train_file_path,sep="@")

/tmp/ipykernel_11545/1922641134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train_df["class"] = new_train_df["class"].map(lambda x:ind2label[x])


                                               tweet               class
0  As a woman you shouldn t complain about cleani...             neither
1  boy dats cold tyga dwn bad for cuffin dat hoe ...  offensive_language
2  Dawg You ever fuck a bitch and she start to cr...  offensive_language
3            G Anderson based she look like a tranny  offensive_language
4  The shit you hear about me might be true or it...  offensive_language


/tmp/ipykernel_11545/1922641134.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train_df["tweet"] = new_train_df["tweet"].map(lambda x:' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|RT"," ",x).split()))


Prepare Train and Validation set

In [8]:
# load all data
all_texts,all_labels = load_partial_data(train_csv_file,n=-1)
# split data in Train and Validation
train_data,val_data = split_for_train(all_texts,all_labels)
x_train,y_train = train_data[0],train_data[1]
x_val,y_val = val_data[0],val_data[1]
# Further split Train data in Test as Original Kaggle Test data does not have labels
val_data,test_data = split_for_train(x_val,y_val,val_split=0.2)
x_val,y_val = val_data[0],val_data[1]
x_test,y_test = test_data[0],test_data[1]
print("Total %d x_train, %d y_train"%(len(x_train),len(y_train)))
print("Total %d x_val, %d y_val"%(len(x_val),len(y_val)))
print("Total %d x_test, %d y_test"%(len(x_test),len(y_test)))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Test_Executions/hate_speech_classification/data/hate_speech_train.csv'

Dump all the data in PKL file

In [3]:
with open(train_pkl,"wb") as f:
    pickle.dump((x_train,y_train),f)
with open(test_pkl,"wb") as f:
    pickle.dump((x_test,y_test),f)
with open(validation_pkl,"wb") as f:
    pickle.dump((x_val,y_val),f)

Load Original Kaggle Test data 

In [8]:
test_df = pd.read_csv(test_file,delimiter=" ::: ")
print(test_df.head())

/tmp/ipykernel_12020/889731494.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_df = pd.read_csv(test_file,delimiter=" ::: ")


   1         Edgar's Lunch (1998)  \
0  2     La guerra de papá (1977)   
1  3  Off the Beaten Track (2010)   
2  4       Meu Amigo Hindu (2015)   
3  5            Er nu zhai (1955)   
4  6           Riddle Room (2016)   

  L.R. Brane loves his life - his car, his apartment, his job, but especially his girlfriend, Vespa. One day while showering, Vespa runs out of shampoo. L.R. runs across the street to a convenience store to buy some more, a quick trip of no more than a few minutes. When he returns, Vespa is gone and every trace of her existence has been wiped out. L.R.'s life becomes a tortured existence as one strange event after another occurs to confirm in his mind that a conspiracy is working against his finding Vespa.  
0  Spain, March 1964: Quico is a very naughty chi...                                                                                                                                                                                                                   